In [1]:
import os 

os.chdir("../..")

from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import numpy as np
from task1.config import ProjectPaths
import pandas as pd
import torch

paths = ProjectPaths()

# === 3. Set device ===
device = "mps" if torch.backends.mps.is_available() else "cpu"

# === 4. Load and preprocess data ===
def load_datasets(path):
    df = pd.read_csv(path, sep='\t')
    df = df[df['label'].isin(['SUBJ', 'OBJ'])].copy()
    df['label'] = df['label'].map({'OBJ': 0, 'SUBJ': 1})
    df = df[['sentence', 'label']]
    return Dataset.from_pandas(df)

train_dataset = load_datasets(paths.english_data_dir / "train_en.tsv")
val_dataset   = load_datasets(paths.english_data_dir / "dev_en.tsv")
test_dataset  = load_datasets(paths.english_data_dir / "dev_test_en.tsv")

W0613 21:50:04.550000 13020 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
dataset = load_dataset("rotten_tomatoes")
train_dataset_stage1 = dataset["train"]
val_dataset_stage1 = dataset["validation"]

small_train_dataset = train_dataset_stage1.select(range(1000))  # Use only 1000 examples for training
small_val_dataset = val_dataset_stage1.select(range(200))

In [4]:
model_name = "microsoft/deberta-v3-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


STAGE 1 FINETUNE

In [3]:
def tokenize_stage1(examples):
    # The text column in this dataset is named 'text'
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

In [5]:
tokenized_train_dataset = small_train_dataset.map(tokenize_stage1, batched=True)
tokenized_val_dataset = small_val_dataset.map(tokenize_stage1, batched=True)

# The 'label' column is already correct, just need to set the format
tokenized_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
tokenized_val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# The Trainer works best when the label column is named 'labels'
tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
tokenized_val_dataset = tokenized_val_dataset.rename_column("label", "labels")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [6]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query_proj", "key_proj", "value_proj", "dense"] 
)

model = get_peft_model(model, lora_config).to(device)

In [7]:
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="macro")["recall"],
    }

In [9]:
# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

trainer.save_model("./results/stage1-rotten-tomatoes/final_checkpoint")

  0%|          | 0/1250 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.5125, 'grad_norm': 3.277414321899414, 'learning_rate': 4.96e-05, 'epoch': 0.04}
{'loss': 0.438, 'grad_norm': 2.9193906784057617, 'learning_rate': 4.92e-05, 'epoch': 0.08}
{'loss': 0.3646, 'grad_norm': 2.579204797744751, 'learning_rate': 4.88e-05, 'epoch': 0.12}
{'loss': 0.276, 'grad_norm': 2.0394837856292725, 'learning_rate': 4.8400000000000004e-05, 'epoch': 0.16}
{'loss': 0.1926, 'grad_norm': 1.5165563821792603, 'learning_rate': 4.8e-05, 'epoch': 0.2}
{'loss': 0.1219, 'grad_norm': 1.0093077421188354, 'learning_rate': 4.76e-05, 'epoch': 0.24}
{'loss': 0.0604, 'grad_norm': 0.6101557612419128, 'learning_rate': 4.72e-05, 'epoch': 0.28}
{'loss': 0.0225, 'grad_norm': 0.2730678915977478, 'learning_rate': 4.6800000000000006e-05, 'epoch': 0.32}
{'loss': 0.0088, 'grad_norm': 0.15851998329162598, 'learning_rate': 4.64e-05, 'epoch': 0.36}
{'loss': 0.0042, 'grad_norm': 0.06501436978578568, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.4}
{'loss': 0.0023, 'grad_norm': 0.053331635892

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 0.00015026680193841457, 'eval_accuracy': 1.0, 'eval_f1_macro': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 14.3622, 'eval_samples_per_second': 13.925, 'eval_steps_per_second': 3.481, 'epoch': 1.0}


...

  0%|          | 0/50 [00:00<?, ?it/s]

{'eval_loss': 1.607583908480592e-05, 'eval_accuracy': 1.0, 'eval_f1_macro': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 13.9267, 'eval_samples_per_second': 14.361, 'eval_steps_per_second': 3.59, 'epoch': 5.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\peft\utils\other.py:611: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f84f143b-c1b7-4ef3-8a64-83b8a85befbf)') - silently ignoring the lookup for the file config.json in microsoft/deberta-v3-base.
  warnings.warn(
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in microsoft/deberta-v3-base - will assume that the vocabulary was not modified.
  warnings.warn(


{'train_runtime': 1715.1648, 'train_samples_per_second': 2.915, 'train_steps_per_second': 0.729, 'train_loss': 0.016182669390528464, 'epoch': 5.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


STAGE 2

In [11]:
def tokenize_fn(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
test_dataset = test_dataset.map(tokenize_fn, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/462 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

In [12]:
from peft import PeftModel

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model = PeftModel.from_pretrained(base_model, "./results/stage1-rotten-tomatoes/final_checkpoint")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.05,
    task_type=TaskType.SEQ_CLS,
    target_modules=["query_proj", "key_proj", "value_proj", "dense"] 
)

model = get_peft_model(model, lora_config).to(device)

In [14]:
# === 8. TrainingArguments ===
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# === 10. Train ===
trainer.train()

# === 11. Evaluate on test set ===
trainer.evaluate(eval_dataset=test_dataset)

  0%|          | 0/2080 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.7158, 'grad_norm': 2.6309142112731934, 'learning_rate': 4.9759615384615386e-05, 'epoch': 0.05}
{'loss': 0.7413, 'grad_norm': 4.365935325622559, 'learning_rate': 4.9519230769230776e-05, 'epoch': 0.1}
{'loss': 0.6976, 'grad_norm': 2.2812647819519043, 'learning_rate': 4.927884615384616e-05, 'epoch': 0.14}
{'loss': 0.6893, 'grad_norm': 1.0032331943511963, 'learning_rate': 4.9038461538461536e-05, 'epoch': 0.19}
{'loss': 0.6935, 'grad_norm': 1.085867166519165, 'learning_rate': 4.8798076923076926e-05, 'epoch': 0.24}
{'loss': 0.665, 'grad_norm': 3.629504442214966, 'learning_rate': 4.855769230769231e-05, 'epoch': 0.29}
{'loss': 0.6473, 'grad_norm': 1.706917405128479, 'learning_rate': 4.8317307692307693e-05, 'epoch': 0.34}
{'loss': 0.6658, 'grad_norm': 3.340214729309082, 'learning_rate': 4.8076923076923084e-05, 'epoch': 0.38}
{'loss': 0.6709, 'grad_norm': 1.4903264045715332, 'learning_rate': 4.783653846153847e-05, 'epoch': 0.43}
{'loss': 0.65, 'grad_norm': 1.477706789970398, 'learning

  0%|          | 0/116 [00:00<?, ?it/s]

d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

  0%|          | 0/116 [00:00<?, ?it/s]

{'eval_loss': 0.7592930197715759, 'eval_accuracy': 0.8008658008658008, 'eval_f1_macro': 0.8008620689655173, 'eval_precision': 0.8024793388429752, 'eval_recall': 0.8022522522522523, 'eval_runtime': 30.9328, 'eval_samples_per_second': 14.936, 'eval_steps_per_second': 3.75, 'epoch': 10.0}
{'train_runtime': 2995.3065, 'train_samples_per_second': 2.771, 'train_steps_per_second': 0.694, 'train_loss': 0.42064572182985455, 'epoch': 10.0}


d:\Master\AN1SEM2\BioNLP\CLEF2025-CheckThat\task1\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/121 [00:00<?, ?it/s]

{'eval_loss': 0.8633573055267334,
 'eval_accuracy': 0.8057851239669421,
 'eval_f1_macro': 0.673611111111111,
 'eval_precision': 0.7854435831180018,
 'eval_recall': 0.6500769857802735,
 'eval_runtime': 33.5153,
 'eval_samples_per_second': 14.441,
 'eval_steps_per_second': 3.61,
 'epoch': 10.0}